In [11]:
import pandas as pd

In [ ]:
train = pd.read_csv("datasets/train.csv.zip", compression="zip", header=0)
test = pd.read_csv("datasets/test.csv.zip", compression="zip", header=0)


In [15]:
train["label"].size


42000